## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-02-23-52-18 +0000,nypost,New club Moss brings five stories of luxury am...,https://nypost.com/2025/12/02/business/new-clu...
1,2025-12-02-23-52-00 +0000,nyt,U.N. Security Council to Visit Syria and Leban...,https://www.nytimes.com/2025/12/02/world/middl...
2,2025-12-02-23-48-54 +0000,nyt,U.S. Pauses Immigration Applications From Nati...,https://www.nytimes.com/2025/12/02/us/politics...
3,2025-12-02-23-47-10 +0000,cbc,Legault calls Miller a 'disgrace to all Quebec...,https://www.cbc.ca/news/canada/montreal/miller...
4,2025-12-02-23-46-42 +0000,nypost,Geezer who terrorized Jewish-owned businesses ...,https://nypost.com/2025/12/02/us-news/geezer-w...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2328/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
46,trump,69
0,new,21
49,he,18
79,up,17
68,administration,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...,221
321,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...,139
349,2025-12-02-00-36-00 +0000,wsj,The White House published a website detailing ...,https://www.wsj.com/politics/policy/new-white-...,135
337,2025-12-02-01-32-00 +0000,wsj,"Officially, the search for a new Federal Reser...",https://www.wsj.com/politics/policy/trump-fed-...,131
18,2025-12-02-23-12-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,129


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,221,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...
144,71,2025-12-02-19-29-00 +0000,wsj,Defense Secretary Pete Hegseth said he didn’t ...,https://www.wsj.com/politics/national-security...
324,62,2025-12-02-03-11-29 +0000,nyt,Kristi Noem Recommends New Travel Ban After Na...,https://www.nytimes.com/2025/12/01/us/politics...
47,60,2025-12-02-22-40-22 +0000,nypost,White House trolls Sabrina Carpenter with her ...,https://nypost.com/2025/12/02/us-news/white-ho...
166,54,2025-12-02-18-23-52 +0000,cbc,"Ex-Honduran president, convicted of traffickin...",https://www.cbc.ca/news/world/us-honduras-hern...
323,47,2025-12-02-03-13-14 +0000,nypost,Trump calls Tennessee special election a ‘big ...,https://nypost.com/2025/12/01/us-news/trump-ca...
321,45,2025-12-02-03-20-00 +0000,wsj,Costco one of the biggest companies to sue the...,https://www.wsj.com/politics/policy/costco-joi...
203,41,2025-12-02-15-55-11 +0000,nypost,Vladimir Putin says he’s ready to go to war wi...,https://nypost.com/2025/12/02/world-news/putin...
213,39,2025-12-02-15-11-10 +0000,nypost,Unmoved Trump admin fires 8 immigration judges...,https://nypost.com/2025/12/02/us-news/8-immigr...
113,38,2025-12-02-20-29-40 +0000,latimes,Saudi Arabia's public investment fund to own a...,https://www.latimes.com/entertainment-arts/bus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
